In [323]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional: Improves WordNet performance

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/kaggle/input/social-media-sentiments-analysis-dataset/sentimentdataset.csv


In [324]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/social-media-sentiments-analysis-dataset/sentimentdataset.csv")

In [325]:
import nltk
import subprocess

nltk.download('wordnet', download_dir='/kaggle/working/')
nltk.download('omw-1.4', download_dir='/kaggle/working/')

command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
subprocess.run(command.split())

nltk.data.path.append('/kaggle/working/')

lemmatizer = WordNetLemmatizer()

train_df['Sentiment'] = train_df['Sentiment'].astype(str).str.strip().str.lower().apply(lemmatizer.lemmatize)
print(train_df['Sentiment'].value_counts())

[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /kaggle/working/...
[nltk_data]   Package omw-1.4 is already up-to-date!
Sentiment
positive                45
joy                     44
excitement              37
contentment             19
neutral                 18
gratitude               18
curiosity               16
serenity                15
happy                   14
nostalgia               11
despair                 11
grief                    9
awe                      9
sad                      9
hopeful                  9
loneliness               9
embarrassed              8
acceptance               8
confusion                8
euphoria                 7
elation                  7
enthusiasm               7
pride                    7
determination            7
regret                   6
frustration              6
ambivalence              6
melancholy               6
nu

In [326]:
unique_values = set()
unique_values.update(train_df['Sentiment'])
unique_values

{'acceptance',
 'accomplishment',
 'admiration',
 'adoration',
 'adrenaline',
 'adventure',
 'affection',
 'amazement',
 'ambivalence',
 'amusement',
 'anger',
 'anticipation',
 'anxiety',
 'appreciation',
 'apprehensive',
 'arousal',
 'artisticburst',
 'awe',
 'bad',
 'betrayal',
 'bitter',
 'bitterness',
 'bittersweet',
 'blessed',
 'boredom',
 'breakthrough',
 'calmness',
 'captivation',
 'celebration',
 'celestial wonder',
 'challenge',
 'charm',
 'colorful',
 'compassion',
 'compassionate',
 'confidence',
 'confident',
 'confusion',
 'connection',
 'contemplation',
 'contentment',
 'coziness',
 'creative inspiration',
 'creativity',
 'culinary adventure',
 'culinaryodyssey',
 'curiosity',
 'darkness',
 'dazzle',
 'desolation',
 'despair',
 'desperation',
 'determination',
 'devastated',
 'disappointed',
 'disappointment',
 'disgust',
 'dismissive',
 'dreamchaser',
 'ecstasy',
 'elation',
 'elegance',
 'embarrassed',
 'emotion',
 'emotionalstorm',
 'empathetic',
 'empowerment',
 'e

In [327]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_md")

ref_words = {
    "Positive": "positive",
    "Negative": "negative",
    "Neutral": "neutral"
}

ref_vectors = {category: nlp(word).vector for category, word in ref_words.items()}

def assign_sentiment_category(sentiment):
    word_vector = nlp(sentiment).vector.reshape(1, -1)
    
    similarities = {}
    for category, ref_vec in ref_vectors.items():
        ref_vec = ref_vec.reshape(1, -1)
        sim = cosine_similarity(word_vector, ref_vec)[0][0]
        similarities[category] = sim

    return max(similarities, key=similarities.get)

train_df['Sentiment'] = train_df['Sentiment'].apply(assign_sentiment_category)

print(train_df['Sentiment'].value_counts())

Sentiment
Positive    364
Negative    290
Neutral      78
Name: count, dtype: int64


In [328]:
!pip install nltk  

import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
import re
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F700-\U0001F77F" 
                           u"\U0001F780-\U0001F7FF"  
                           u"\U0001F800-\U0001F8FF" 
                           u"\U0001F900-\U0001F9FF" 
                           u"\U0001FA00-\U0001FA6F" 
                           u"\U0001FA70-\U0001FAFF" 
                           u"\U00002702-\U000027B0" 
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)

def split_and_remove_stopwords(text):
    text = emoji_pattern.sub(r'', text)
    tokens = text.split()

    filtered_tokens = [
        stemmer.stem(lemmatizer.lemmatize(word.lower().rstrip(string.punctuation)))
        for word in tokens if stemmer.stem(word.lower().rstrip(string.punctuation)) not in stop_words
    ]

    return filtered_tokens

train_df['Text'] = train_df['Text'].apply(split_and_remove_stopwords)

print(train_df['Text'])

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0                             [enjoy, beauti, day, park]
1                      [traffic, wa, terribl, thi, morn]
2                                [finish, amaz, workout]
3                       [excit, upcom, weekend, getaway]
4                     [tri, new, recip, dinner, tonight]
5                      [feel, grate, littl, thing, life]
6          [raini, day, call, cozi, blanket, hot, cocoa]
7                        [new, movi, releas, must-watch]
8                        [polit, discuss, heat, timelin]
9                       [miss, summer, vibe, beach, day]
10                     [publish, new, blog, post, check]
11       

In [330]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Assume train_df is already loaded with at least the columns "Text" and "Sentiment"
# For example, train_df might look like:
#    Text                                               Sentiment
# 0  "Enjoying a beautiful day at the park!"            Positive
# 1  "Traffic was terrible this morning."               Negative
# ...
from sklearn.feature_extraction.text import TfidfVectorizer

# Ensure that each document is a string (join list back into a sentence)
train_df['Text'] = train_df['Text'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

# Use TF-IDF Vectorizer (limit features to 900 for better performance)
vectorizer = TfidfVectorizer(max_features=900)
X = vectorizer.fit_transform(train_df['Text']).toarray()

print("TF-IDF feature matrix shape:", X.shape)


# -----------------------------
# Step 2. Encode Sentiment Labels
# -----------------------------
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_df['Sentiment'])
print("Number of classes:", len(np.unique(y)))

# -----------------------------
# Step 3. K-Fold Cross Validation
# -----------------------------
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
accuracies = []
fold = 1

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # -----------------------------
    # Step 4. Build the Neural Network Model
    # -----------------------------
    model = Sequential([
        Input(shape=(X.shape[1],)),
        Dense(2048),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(512),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(len(np.unique(y)), activation='softmax')
    ])
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    
    # Learning rate scheduler to reduce LR when validation loss plateaus.
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                     verbose=1, min_lr=1e-6)
    
    # -----------------------------
    # Step 5. Train the Model
    # -----------------------------
    model.fit(X_train, y_train, epochs=100, batch_size=64, 
              validation_data=(X_test, y_test), callbacks=[lr_scheduler], verbose=1)
    
    # Evaluate the model on the test fold.
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Fold {fold} Accuracy: {accuracy:.4f}")
    accuracies.append(accuracy)
    fold += 1

# -----------------------------
# Step 6. Print the Average Accuracy
# -----------------------------
print(f"Mean Accuracy over {n_splits} folds: {np.mean(accuracies):.4f}")

TF-IDF feature matrix shape: (732, 900)
Number of classes: 3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.3635 - loss: 1.8176 - val_accuracy: 0.4762 - val_loss: 1.0568 - learning_rate: 0.0010
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.4598 - loss: 1.4035 - val_accuracy: 0.5442 - val_loss: 1.0211 - learning_rate: 0.0010
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.5130 - loss: 1.1344 - val_accuracy: 0.4694 - val_loss: 0.9912 - learning_rate: 0.0010
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.6150 - loss: 0.9228 - val_accuracy: 0.4626 - val_loss: 0.9684 - learning_rate: 0.0010
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.7017 - loss: 0.7472 - val_accuracy: 0.5782 - val_loss: 0.9340 - learning_rate: 0.0010
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7837 - loss: 0.5246 - val_accuracy: 0.5510 - val_loss: 0.9073 - learning_rate: 0.0010
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.8458 - loss: 0.3679 - 